<a href="https://colab.research.google.com/github/katiacardoso/EMAP_Chatbot/blob/main/chatbot_emap_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Teste Implementação: Chatbot com Bert

Importação de bibliotecas

In [ ]:
import pandas as pd
#!pip install transformers
from transformers import BertForQuestionAnswering, BertTokenizer,  pipeline
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch




Parte do código que é responsável por carregar um modelo BERT pré-treinado e seu tokenizador associado. O modelo BERT será usado para responder a perguntas com base em um contexto fornecido. Essa configuração permite que o chatbot compreenda e responda às perguntas dos usuários com base no conhecimento prévio adquirido durante o treinamento do modelo BERT.

- O tokenizador é usado para dividir o texto em unidades menores chamadas tokens, o que é necessário para processar texto com modelos de linguagem como o BERT





In [ ]:

# Carregue o modelo BERT pré-treinado e o tokenizador
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

# Carregue o arquivo CSV com perguntas e respostas
df = pd.read_csv("perguntas_respostas.csv")


Na **Tokenização da Pergunta do Usuário** é feita uma conversão da pergunta em uma sequência de ID's de tokens que o modelo BERT pode entender.



In [ ]:

# Função para responder a uma pergunta com base na similaridade
def chatbot(question):

    # Tokenização da Pergunta do Usuário
    input_ids = tokenizer.encode(question, add_special_tokens=True, return_tensors="pt")

    # Pipeline que usa o modelo BERT para responder à pergunta
    question_answering = pipeline("question-answering", model=model, tokenizer=tokenizer)

    # Encontra a pergunta mais similar no DataFrame
    best_matching_question = None
    best_similarity = 0.1
    for index, row in df.iterrows():
        context = row['Pergunta']
        result = question_answering(question=question, context=context)
        if result['score'] > best_similarity:
            best_similarity = result['score']
            best_matching_question = row['Pergunta']

    if best_matching_question:
        answer = df[df['Pergunta'] == best_matching_question]['Resposta'].values[0]
    else:
        answer = "Desculpe, não entendi a pergunta."

    return answer



In [ ]:
# Inicie uma conversa com o chatbot
while True:
    user_input = input("Você: ")
    if user_input.lower() == "sair":
        print("Chatbot: Adeus! Até logo.")
        break
    response = chatbot(user_input)
    print("Chatbot:", response)

Você: Como chegar no porto?
Chatbot: As informações sobre tarifas portuárias estão disponíveis em www.portodoitaqui.com/porto-do-itaqui/tarifas-portuarias.
Você: como chegar ao porto?
Chatbot: Confira aqui o mapa de localização e seja bem-vindo! http://www.portodoitaqui.ma.gov.br/porto-do-itaqui/como-chegar 
Você: Sair
Chatbot: Adeus! Até logo.
